In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [3]:
csv_1 = "../Resources/crime_data.csv"
crime_df = pd.read_csv(csv_1)
crime_df.head()

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,200,1778,3609,4995,13791,3543,464,318416,29,510
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,38,165,662,1482,1753,189,28,49746,5,35
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,2,5,119,82,184,12,2,7629,17,3
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,3,1,2,5,4,4,0,412,48,261
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,4,17,368,149,494,60,0,27083,22,31


In [4]:
crime_df[['county','state']]=crime_df['county_name'].str.split(", ", expand=True)
crime_df['crime_rate_per_100000']=crime_df['crime_rate_per_100000'].map("{:.0f}".format)
crime_df.head()

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY,county,state
0,"St. Louis city, MO",1792,1,1,4,1612,318667,318667,15,15,...,3609,4995,13791,3543,464,318416,29,510,St. Louis city,MO
1,"Crittenden County, AR",1755,2,1,4,130,50717,50717,4,4,...,662,1482,1753,189,28,49746,5,35,Crittenden County,AR
2,"Alexander County, IL",1665,3,1,4,604,8040,8040,2,2,...,119,82,184,12,2,7629,17,3,Alexander County,IL
3,"Kenedy County, TX",1456,4,1,4,2681,444,444,1,1,...,2,5,4,4,0,412,48,261,Kenedy County,TX
4,"De Soto Parish, LA",1447,5,1,4,1137,26971,26971,3,3,...,368,149,494,60,0,27083,22,31,De Soto Parish,LA


In [5]:
crime_df.columns

Index(['county_name', 'crime_rate_per_100000', 'index', 'EDITION', 'PART',
       'IDNO', 'CPOPARST', 'CPOPCRIM', 'AG_ARRST', 'AG_OFF', 'COVIND', 'INDEX',
       'MODINDX', 'MURDER', 'RAPE', 'ROBBERY', 'AGASSLT', 'BURGLRY', 'LARCENY',
       'MVTHEFT', 'ARSON', 'population', 'FIPS_ST', 'FIPS_CTY', 'county',
       'state'],
      dtype='object')

In [6]:

crime = crime_df.drop(['county_name', 'index', 'EDITION', 'PART', 'IDNO', 'CPOPARST', 'CPOPCRIM', 'AG_ARRST', 'AG_OFF', 'COVIND',
              'INDEX', 'MODINDX', 'MURDER', 'RAPE', 'ROBBERY', 'AGASSLT', 'BURGLRY', 'LARCENY', 'MVTHEFT',
              'ARSON', 'FIPS_ST', 'FIPS_CTY'], axis = 1)
crime_data_df = crime.rename(columns={"crime_rate_per_100000":"crime rate"})
# crime_data_df['county'].value_counts()
# state_count = crime_data_df['state'].value_counts()
crime_data_df["crime rate"] = crime_data_df.loc[:, "crime rate"].astype(float)
crime_data_df.head()

,crime rate,population,county,state
0,1792.0,318416,St. Louis city,MO
1,1755.0,49746,Crittenden County,AR
2,1665.0,7629,Alexander County,IL
3,1456.0,412,Kenedy County,TX
4,1447.0,27083,De Soto Parish,LA


In [7]:
# crime_data_df.loc[crime_data_df['state']=="CA", :]

In [8]:
county_crime = crime_data_df.groupby(['state'])
print(county_crime)
county_crime.count().head()

,crime rate,population,county
state,,,
AK,23,23,23
AL,67,67,67
AR,75,75,75
AZ,15,15,15
CA,58,58,58


In [9]:
state_rate = county_crime['crime rate'].sum()/county_crime['county'].count()
state_population = county_crime['population'].sum()

In [10]:
state_summary = pd.DataFrame({"State crime rate": state_rate,
                            "Population": state_population})
state_summary['State crime rate'] = state_summary['State crime rate'].map("{:.0f}".format)
state_summary.head()

,State crime rate,Population
state,,
AK,289,718165
AL,386,4833722
AR,335,2959373
AZ,353,6626624
CA,416,38332521


In [11]:
csv_2 = r"../Resources/Police_Fatalities.csv"
police_df = pd.read_csv(csv_2, encoding='utf-8')
police_df.head()

,UID,Name,Age,Gender,Race,Date,City,State,Manner_of_death,Armed,Mental_illness,Flee
0,133,Karen O. Chin,44.0,Female,Asian,5/4/00,Alameda,CA,Shot,NaN,False,False
1,169,Chyraphone Komvongsa,26.0,Male,Asian,6/2/00,Fresno,CA,Shot,NaN,False,False
2,257,Ming Chinh Ly,36.0,Male,Asian,8/13/00,Rosemead,CA,Shot,Gun,False,False
3,483,Kinh Quoc Dao,29.0,Male,Asian,2/9/01,Valley Glen,CA,Shot,Gun,False,False
4,655,Vanpaseuth Phaisouphanh,25.0,Male,Asian,6/10/01,Riverside,CA,Shot,Knife,False,False


In [12]:
police_df['Race'].unique()

array(['Asian', 'Black', 'Hispanic', 'Other', 'Native', 'White', nan],
      dtype=object)

In [13]:
# Getting the number of police fatalities by state
fatalities_no = police_df['State'].value_counts()
fatalities = pd.DataFrame(fatalities_no)
fatalities.head()

#Getting the number of fatalities by Gender by each state
male = police_df.loc[police_df['Gender']=="Male", :]
female = police_df.loc[police_df['Gender']=="Female", :]
male_no = male.groupby('State')['Gender'].count()
female_no = female.groupby('State')['Gender'].count()
# gender = pd.DataFrame({"Female": female_no, "Male": male_no})

# Getting the number of fatalities by Race by each state
asian = police_df.loc[police_df['Race']=="Asian", :]
asian_no = asian.groupby('State')['Race'].count()
asian_no
black = police_df.loc[police_df['Race']=="Black", :]
black_no = black.groupby('State')['Race'].count()
black_no
hispanic = police_df.loc[police_df['Race']=="Hispanic", :]
hispanic_no = hispanic.groupby('State')['Race'].count()
hispanic_no
other = police_df.loc[police_df['Race']=="Other", :]
other_no = other.groupby('State')['Race'].count()
other_no
native = police_df.loc[police_df['Race']=="Native", :]
native_no = native.groupby('State')['Race'].count()
native_no
white = police_df.loc[police_df['Race']=="White", :]
white_no = white.groupby('State')['Race'].count()
white_no

# race = pd.DataFrame({"Asian": asian_no, "Black": black_no, "Hispanic": hispanic_no, "Other": other_no,
#                         "Native": native_no, "White": white_no})

# Generating the DataFrame
state_fatalities = pd.DataFrame({"Nr. Fatalities":fatalities_no, "Male": male_no, "Female": female_no,
                                "Asian": asian_no, "Black": black_no, "Hispanic": hispanic_no, "Other": other_no,
                        "Native": native_no, "White": white_no})
state_fatalities.fillna(0, inplace=True)
state_fatalities["Female"]=state_fatalities["Female"].astype(int)
state_fatalities["Asian"]=state_fatalities["Asian"].astype(int)
state_fatalities["Black"]=state_fatalities["Black"].astype(int)
state_fatalities["Hispanic"]=state_fatalities["Hispanic"].astype(int)
state_fatalities["Other"]=state_fatalities["Other"].astype(int)
state_fatalities["Native"]=state_fatalities["Native"].astype(int)
state_fatalities.head()

,Nr. Fatalities,Male,Female,Asian,Black,Hispanic,Other,Native,White
AK,73,68,5,0,3,1,1,12,16
AL,326,306,20,1,80,7,0,0,98
AR,70,69,1,0,16,0,0,0,36
AZ,585,550,32,1,22,117,1,23,170
CA,2913,2771,141,97,420,883,24,20,580


In [14]:
# Getting the number of police fatalities by city

by_city = police_df['City'].value_counts()
by_city.head()

male_nr = male.groupby('City')['Gender'].count()
female_nr = female.groupby('City')['Gender'].count()

asian_nr = asian.groupby('City')['Race'].count()
black_nr = black.groupby('City')['Race'].count()
hispanic_nr = hispanic.groupby('City')['Race'].count()
other_nr = other.groupby('City')['Race'].count()
native_nr = native.groupby('City')['Race'].count()
white_nr = white.groupby('City')['Race'].count()

In [15]:
#Creating the DF by cities 
city_fatalities = pd.DataFrame({"Nr. Fatalities":by_city, "Male": male_nr, "Female": female_nr,
                                "Asian": asian_nr, "Black": black_nr, "Hispanic": hispanic_nr, "Other": other_nr,
                        "Native": native_nr, "White": white_nr})
city_fatalities.fillna(0, inplace=True)
city_fatalities["Male"]=city_fatalities["Male"].astype(int)
city_fatalities["Female"]=city_fatalities["Female"].astype(int)
city_fatalities["Asian"]=city_fatalities["Asian"].astype(int)
city_fatalities["Black"]=city_fatalities["Black"].astype(int)
city_fatalities["Hispanic"]=city_fatalities["Hispanic"].astype(int)
city_fatalities["Other"]=city_fatalities["Other"].astype(int)
city_fatalities["Native"]=city_fatalities["Native"].astype(int)
city_fatalities["White"]=city_fatalities["White"].astype(int)
city_fatalities.head(10)


,Nr. Fatalities,Male,Female,Asian,Black,Hispanic,Other,Native,White
"""El Paso, Cuidad Juarez""",1,1,0,0,0,1,0,0,0
"""Grand Boulevard, Chicago""",1,1,0,0,1,0,0,0,0
"""South Shore, Chicago""",1,1,0,0,1,0,0,0,0
"""West Town, Chicago""",1,1,0,0,0,0,0,0,0
3524 N. Kingshighway Blvd.,1,1,0,0,1,0,0,0,0
Aberdeen,1,1,0,0,1,0,0,0,0
Abilene,6,6,0,0,1,1,0,0,3
Abingdon,2,2,0,0,0,0,0,0,1
Abita Springs,1,1,0,0,0,0,0,0,1
Acampo,1,1,0,0,0,1,0,0,0


In [16]:
connection_string = "jssdoohk:s0CPRxtjMiWM-HePKLlITgViptVMZ6hQ@drona.db.elephantsql.com:5432/jssdoohk"
engine = create_engine(f'postgresql://{connection_string}')

In [22]:
# Confirm tables
engine.table_names()

['state', 'fatalities', 'cities']

In [18]:
state_summary.to_sql(name='state', con=engine, if_exists='append', index=True)

In [19]:
state_fatalities.to_sql(name='fatalities', con=engine, if_exists='replace', index=True)

In [21]:
city_fatalities.to_sql(name='cities', con=engine, if_exists='append', index=True)